In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow

tensorflow.__version__

'2.6.0'

In [4]:
train_dir = 'train' 
valid_dir = 'validation'

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 60, 
                                   shear_range = 0.2,
                                   zoom_range = 0.1, 
                                   horizontal_flip=True,
                                   fill_mode = 'nearest')
valid_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
                  train_dir,
                  target_size = (275,205),
                  batch_size = 32, 
                  class_mode = 'categorical') 


valid_generator = valid_datagen.flow_from_directory(
                  valid_dir,
                  target_size = (275,205),
                  batch_size = 32,
                  class_mode = 'categorical')

Found 12000 images belonging to 6 classes.
Found 4800 images belonging to 6 classes.


In [3]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights = 'imagenet',
                  include_top = False,
                  input_shape = (275,205, 3))

In [7]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 275, 205, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 275, 205, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 275, 205, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 137, 102, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 137, 102, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 137, 102, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 68, 51, 128)       0     

In [5]:
print('conv_base 동결 전 훈련 가능 가중치의 종류:', len(conv_base.trainable_weights))

conv_base 동결 전 훈련 가능 가중치의 종류: 26


In [6]:
set_trainable = False

for layer in conv_base.layers:
    if layer.name == 'block4_conv1':
        set_trainable = True
        
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [9]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 275, 205, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 275, 205, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 275, 205, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 137, 102, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 137, 102, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 137, 102, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 68, 51, 128)       0     

In [10]:
from tensorflow.keras import models, layers

model = models.Sequential(name = 'CNN_VGG16')
model.add(conv_base)

model.add(layers.Flatten())
model.add(layers.Dropout(0.4))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(6, activation = 'softmax'))

In [7]:
print('conv_base 동결 후 훈련 가능 가중치의 종류:', len(conv_base.trainable_weights))

conv_base 동결 후 훈련 가능 가중치의 종류: 12


In [12]:
model.summary()

Model: "CNN_VGG16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 8, 6, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 24576)             0         
_________________________________________________________________
dropout (Dropout)            (None, 24576)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               6291712   
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1542      
Total params: 21,007,942
Trainable params: 19,272,454
Non-trainable params: 1,735,488
_________________________________________________________________


In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint

filename = '/{epoch:02d}-{val_loss:.5f}.h5'

checkpoint = ModelCheckpoint(filename,          
                             monitor='val_loss',  
                             verbose=1,           
                             save_best_only=True,  
                             mode='min'          
                            )

In [16]:
from tensorflow.keras import optimizers

model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizers.Adam(lr = 0.00001),
              metrics = ['accuracy'])

In [17]:
%%time

face = model.fit(train_generator,
                epochs = 50,
                validation_data = valid_generator,
                validation_steps = 150,
                callbacks = [checkpoint])

Epoch 1/50


2021-11-15 09:25:37.948942: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-11-15 09:25:38.679461: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7


375/375 [==============================] - ETA: 0s - loss: 1.4744 - accuracy: 0.4069
Epoch 00001: val_loss improved from inf to 1.10715, saving model to /home/lab02/01-1.10715.h5
375/375 [==============================] - 319s 851ms/step - loss: 1.4744 - accuracy: 0.4069 - val_loss: 1.1071 - val_accuracy: 0.5785
Epoch 2/50
375/375 [==============================] - ETA: 0s - loss: 1.1097 - accuracy: 0.5789
Epoch 00002: val_loss improved from 1.10715 to 0.98747, saving model to /home/lab02/02-0.98747.h5
375/375 [==============================] - 307s 817ms/step - loss: 1.1097 - accuracy: 0.5789 - val_loss: 0.9875 - val_accuracy: 0.6252
Epoch 3/50
375/375 [==============================] - ETA: 0s - loss: 0.9767 - accuracy: 0.6400
Epoch 00003: val_loss improved from 0.98747 to 0.92965, saving model to /home/lab02/03-0.92965.h5
375/375 [==============================] - 307s 818ms/step - loss: 0.9767 - accuracy: 0.6400 - val_loss: 0.9296 - val_accuracy: 0.6546
Epoch 4/50
375/375 [========

KeyboardInterrupt: 